# A Transformer-based recommendation system


In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
from src.dataset import RatingDataset
from src import utils
from torch.utils.data import DataLoader
from torch import nn
import math
import torch

### (1) Load Dataset

In [3]:
df_train = pd.read_parquet("./artifacts/train_data.parquet")
df_test = pd.read_parquet("./artifacts/test_data.parquet")

In [4]:
train_dataset = RatingDataset(data=df_train) 

In [5]:
loader = DataLoader(train_dataset,batch_size=12,shuffle=True)

In [6]:
for inputs in loader:
    break


## (2) Model Config

In [7]:
# inputs

In [8]:
age_group_id_map_dict = utils.open_object("./artifacts/age_group_id_map_dict.pkl")

movie_id_map_dict = utils.open_object("./artifacts/movie_id_map_dict.pkl")

occupation_id_map_dict = utils.open_object("./artifacts/occupation_id_map_dict.pkl")

sex_id_map_dict = utils.open_object("./artifacts/sex_id_map_dict.pkl")

user_id_map_dict = utils.open_object("./artifacts/user_id_map_dict.pkl")
# genres_map_dict = utils.open_object("./artifacts/genres_map_dict.pkl")

In [9]:
rating_min_max_scaler = utils.open_object("./artifacts/rating_min_max_scaler.pkl")

In [10]:
num_user = len(user_id_map_dict)
num_movie = len(movie_id_map_dict)
num_occupation = len(occupation_id_map_dict)
num_age_group = len(age_group_id_map_dict)
# num_genre = len(genres_map_dict)

In [11]:
embed_configs = {}
EMED_DIM=32
sequence_length=4
embed_configs['user']={"embed_dim":EMED_DIM,"num_embed":num_user}
embed_configs['movie']={"embed_dim":EMED_DIM,"num_embed":num_movie}
embed_configs['occupation']={"embed_dim":EMED_DIM,"num_embed":num_occupation}
embed_configs['age_group']={"embed_dim":EMED_DIM,"num_embed":num_age_group}
embed_configs['position'] = {"embed_dim":EMED_DIM,"num_embed":sequence_length}

In [12]:
config_dict={}
config_dict['embed_configs'] = embed_configs
config_dict['transformer_num_layer']=3
config_dict['dropout']=0.2

In [13]:
class Config:
    def __init__(self, dictionary):
        for key, value in dictionary.items():
            setattr(self, key, value)

In [14]:
config = Config(dictionary=config_dict)

In [15]:
config.embed_configs

{'user': {'embed_dim': 32, 'num_embed': 6041},
 'movie': {'embed_dim': 32, 'num_embed': 3884},
 'occupation': {'embed_dim': 32, 'num_embed': 22},
 'age_group': {'embed_dim': 32, 'num_embed': 8},
 'position': {'embed_dim': 32, 'num_embed': 4}}

## Modeling

### (1) Create Embedding Layer

In [16]:
class self:
    pass

In [17]:
self.config = config

In [18]:
embed_configs = self.config.embed_configs

embedding_layers = []
for name,embed_config in embed_configs.items():
    embed_dim = embed_config['embed_dim']
    num_embed = embed_config['num_embed']
    embeding_layer = nn.Embedding(num_embeddings=num_embed, embedding_dim=embed_dim)
    nn.init.xavier_uniform_(embeding_layer.weight)
    embedding_layers.append([name,embeding_layer])

self.embedding_layers = nn.ModuleDict(embedding_layers)

In [19]:
self.embedding_layers

ModuleDict(
  (user): Embedding(6041, 32)
  (movie): Embedding(3884, 32)
  (occupation): Embedding(22, 32)
  (age_group): Embedding(8, 32)
  (position): Embedding(4, 32)
)

In [20]:
inputs.keys()

dict_keys(['user_id_index', 'movie_sequence', 'rating_sequence', 'sex', 'occupation_index', 'age_group_index', 'target_rating', 'target_movie'])

### (2) Embed Movie

In [21]:
inputs['target_movie']

tensor([2462,  171, 1179,  471, 3382, 2804, 1268,  159, 3549, 3564, 1897, 2040])

In [22]:
target_movie_embedding = self.embedding_layers['movie'](inputs['target_movie'])

In [23]:
target_movie_embedding.shape

torch.Size([12, 32])

In [24]:
movie_sequence_embedding  = self.embedding_layers['movie'](inputs['movie_sequence'])

In [25]:
movie_sequence_embedding.shape

torch.Size([12, 4, 32])

### (4) Position Embedding

In [26]:
self.config.embed_configs['position']

{'embed_dim': 32, 'num_embed': 4}

In [27]:
positions = torch.arange(self.config.embed_configs['position']['num_embed'])

In [28]:
position_embedding = self.embedding_layers['position'](positions)

In [29]:
position_embedding.shape

torch.Size([4, 32])

### (5) Sequence Rating

In [30]:
rating_sequence = inputs['rating_sequence']

In [31]:
rating_sequence.shape

torch.Size([12, 4])

In [32]:
inputs.keys()

dict_keys(['user_id_index', 'movie_sequence', 'rating_sequence', 'sex', 'occupation_index', 'age_group_index', 'target_rating', 'target_movie'])

### (5) Embedding User

In [33]:
self.embedding_layers

ModuleDict(
  (user): Embedding(6041, 32)
  (movie): Embedding(3884, 32)
  (occupation): Embedding(22, 32)
  (age_group): Embedding(8, 32)
  (position): Embedding(4, 32)
)

In [34]:
sex_input = inputs['sex']
user_embedding = self.embedding_layers['user'](inputs['user_id_index'])
occupation_embedding = self.embedding_layers['occupation'](inputs['occupation_index'])
age_group_embedding = self.embedding_layers['age_group'](inputs['age_group_index'])

### (6) User Behavior Sequence Input

In [35]:
position_embedding.shape

torch.Size([4, 32])

In [36]:
batch_size = movie_sequence_embedding.shape[0]

In [37]:
batch_position_embedding = torch.stack([position_embedding.clone() for _ in range(batch_size)])
batch_position_embedding.shape

torch.Size([12, 4, 32])

In [38]:
movie_sequence_embedding.shape

torch.Size([12, 4, 32])

In [39]:
movie_pos_seq_embedding = torch.concat([movie_sequence_embedding,batch_position_embedding],dim=-1)

In [40]:
movie_pos_seq_embedding.shape

torch.Size([12, 4, 64])

In [41]:
## cross with rating

In [42]:
rating_sequence.unsqueeze(-1).shape

torch.Size([12, 4, 1])

In [43]:
torch.mul(movie_pos_seq_embedding,rating_sequence.unsqueeze(-1)).shape

torch.Size([12, 4, 64])

In [44]:
movie_pos_rating_seq_embedding = torch.mul(movie_pos_seq_embedding,rating_sequence.unsqueeze(-1))

### (6) Transformer for sequence features

In [45]:
%%writefile ./src/transformer_layer.py

import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerBlock(nn.Module):
    def __init__(self, input_size, output_size, num_heads, dropout_rate):
        super(TransformerBlock, self).__init__()

        self.multihead_attention = nn.MultiheadAttention(input_size, num_heads)
        self.layer_norm1 = nn.LayerNorm(input_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(input_size, 4*input_size),
            nn.ReLU(),
            nn.Linear(4*input_size, output_size),
            nn.Dropout(dropout_rate)
        )
        self.layer_norm2 = nn.LayerNorm(output_size)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        # Multi-head Attention
        attn_output, _ = self.multihead_attention(x, x, x)
        x = self.layer_norm1(x + attn_output)

        # Feed-Forward Network
        ff_output = self.feed_forward(x)
        x = self.layer_norm2(x + ff_output)
        x = self.dropout(x)
        return x

class TransformerLayer(nn.Module):
    def __init__(self, d_model, num_heads=8, dropout_rate=0.2, num_layers=3):
        super(TransformerLayer, self).__init__()

        self.transformer_blocks = nn.ModuleList([
            TransformerBlock(d_model, d_model, num_heads, dropout_rate)
            for _ in range(num_layers)
        ])

    def forward(self, x):
        for transformer_block in self.transformer_blocks:
            x = transformer_block(x)
        return x


Writing ./src/transformer_layer.py


In [47]:
src = torch.rand(10, 32, 64)

In [48]:
from src.transformer_layer import TransformerLayer

In [49]:
transformer_layer = TransformerLayer(d_model=64,num_heads=8, dropout_rate=0.2, num_layers=3)

In [50]:
transformer_layer(src).shape

torch.Size([10, 32, 64])

In [51]:
movie_pos_rating_seq_embedding.shape

torch.Size([12, 4, 64])

In [52]:
seq_transformer_output = transformer_layer(movie_pos_rating_seq_embedding)

In [53]:
seq_transformer_output.shape

torch.Size([12, 4, 64])

In [54]:
seq_transformer_flatten_output = seq_transformer_output.view(12,-1)

In [55]:
seq_transformer_flatten_output.shape

torch.Size([12, 256])

### (7) Concat other Features

In [56]:
inputs.keys()

dict_keys(['user_id_index', 'movie_sequence', 'rating_sequence', 'sex', 'occupation_index', 'age_group_index', 'target_rating', 'target_movie'])

In [57]:
sex_feature = inputs['sex']
user_embedding = self.embedding_layers['user'](inputs['user_id_index'])
occupation_embedding = self.embedding_layers['occupation'](inputs['occupation_index'])
age_group_embedding = self.embedding_layers['age_group'](inputs['age_group_index'])

In [58]:
target_movie_embedding.shape

torch.Size([12, 32])

In [59]:
sex_feature = sex_feature.unsqueeze(-1)
sex_feature.shape

torch.Size([12, 1])

In [60]:
sex_cross_feature = torch.mul(sex_feature,target_movie_embedding)

In [61]:
user_embedding.shape

torch.Size([12, 32])

In [62]:
user_embedding_cross = torch.mul(user_embedding,target_movie_embedding)

In [63]:
occupation_embedding_cross = torch.mul(occupation_embedding,target_movie_embedding,)

In [64]:
age_group_embedding_cross = torch.mul(age_group_embedding,target_movie_embedding)

In [65]:
user_features = torch.concat([sex_feature,user_embedding,occupation_embedding,age_group_embedding],dim=-1)

In [66]:
user_cross_features = torch.concat([sex_cross_feature,user_embedding_cross,occupation_embedding_cross,age_group_embedding_cross],dim=-1)

In [71]:
user_inputs_features = torch.concat([user_features,user_cross_features,target_movie_embedding],axis=1)

In [72]:
user_inputs_features.shape

torch.Size([12, 257])

#### MLP Layer 

In [130]:
%%writefile ./src/mlp_layer.py

import torch.nn as nn

class MLP(nn.Module):
    def __init__(self, dropout=0.2, hidden_units=[512, 256,128]):
        super(MLP, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.layers = nn.ModuleList()
        for i in range(len(hidden_units) - 1):
            self.layers.append(nn.Linear(hidden_units[i], hidden_units[i + 1]))
            self.layers.append(nn.LeakyReLU())
            self.layers.append(nn.Dropout(p=dropout))
        self.fc = nn.Linear(hidden_units[-1],1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        logits = self.fc(x)
        output = self.sigmoid(logits)
        return output


Overwriting ./src/mlp_layer.py


In [131]:
mlp_input_features = torch.concat([user_inputs_features,seq_transformer_flatten_output],axis=1)

In [132]:
mlp_input_features.shape

torch.Size([12, 513])

In [133]:
from src.mlp_layer import MLP

In [134]:
mlp = MLP(dropout=0.2, hidden_units=[513, 256,128])

In [135]:
mlp

MLP(
  (dropout): Dropout(p=0.2, inplace=False)
  (layers): ModuleList(
    (0): Linear(in_features=513, out_features=256, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): LeakyReLU(negative_slope=0.01)
    (5): Dropout(p=0.2, inplace=False)
  )
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [136]:
outputs = mlp(mlp_input_features)

In [137]:
outputs

tensor([[0.4577],
        [0.5000],
        [0.4769],
        [0.4764],
        [0.4517],
        [0.4620],
        [0.4705],
        [0.4790],
        [0.4628],
        [0.4460],
        [0.4561],
        [0.4562]], grad_fn=<SigmoidBackward0>)

In [142]:
loss_func = torch.nn.BCELoss()

In [143]:
loss_func(outputs,inputs['target_rating'].view(-1,1))

tensor(0.7010, grad_fn=<BinaryCrossEntropyBackward0>)